In [22]:
import torch
from torch.utils import data
from torchvision import models, transforms
import os.path as osp
import glob
from PIL import Image
from tqdm import tqdm

In [2]:
torch.manual_seed(0)

In [3]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

In [4]:
root_path = "./data/hymenoptera_data"
target_path = osp.join(root_path, "train")

In [5]:
def make_data_list(phase="train"):
    root_path = "./data/hymenoptera_data"
    target_path = osp.join(root_path, phase)
    
    return glob.glob(osp.join(target_path, "*/*.jpg"))

In [6]:
train_list = make_data_list("train")
val_list = make_data_list("val")

In [7]:
class MyDataset(data.Dataset):
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.phase)
        
        if img_path.split("/")[-2] == "ants":
            label = 0
        elif img_path.split("/")[-2] == "bees":
            label = 1
        
        return img_transformed, label

In [8]:
train_dataset = MyDataset(file_list=train_list, transform=ImageTransform(224, (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), phase='train')
val_dataset = MyDataset(file_list=val_list, transform=ImageTransform(224, (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), phase='val')

In [9]:
index = 0

print(train_dataset.__len__())

img, label = train_dataset.__getitem__(index)
print(img.size())
print(label)

243
torch.Size([3, 224, 224])
1


In [25]:
batch_size = 128

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [11]:
batch_iterator = iter(dataloaders_dict["train"])
inputs, labels = next(batch_iterator)
print(inputs.shape)
print(labels)

torch.Size([32, 3, 224, 224])
tensor([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 1])


# Network

In [14]:
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
net.classifier[6] = torch.nn.Linear(in_features=4096, out_features=2)
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
net = net.train()

# Loss

In [17]:
criterion = torch.nn.CrossEntropyLoss()

# Optimizer

In [18]:
optimizer = torch.optim.SGD(params=net.parameters(), lr=0.001, momentum=0.9)

In [19]:
params_to_update = []

update_params_name = ["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
    if name in update_params_name:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

print(params_to_update)

classifier.6.weight
classifier.6.bias
[Parameter containing:
tensor([[ 0.0081,  0.0090, -0.0032,  ...,  0.0116,  0.0040, -0.0135],
        [-0.0009, -0.0058, -0.0099,  ..., -0.0091, -0.0089, -0.0046]],
       requires_grad=True), Parameter containing:
tensor([-0.0144,  0.0010], requires_grad=True)]


# Training - Transfer Learning

In [23]:
def train_model(net, dataloader_dict, num_epochs):
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch+1, num_epochs))
        print("-------------")
        
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
            else:
                net.eval()
            
            epoch_loss = 0.0
            epoch_corrects = 0
            
            if (epoch == 0) and (phase == "train"):
                continue
            
            for inputs, labels in tqdm(dataloader_dict[phase]):
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                    
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))

In [26]:
num_epochs = 10
train_model(net, dataloaders_dict, num_epochs)

Epoch 1/10
-------------


100%|██████████| 2/2 [00:16<00:00,  8.34s/it]


val Loss: 0.1548 Acc: 0.9477
Epoch 2/10
-------------


100%|██████████| 2/2 [00:30<00:00, 15.29s/it]


train Loss: 0.1352 Acc: 0.9712


...

val Loss: 0.1096 Acc: 0.9608
Epoch 10/10
-------------


100%|██████████| 2/2 [00:22<00:00, 11.43s/it]


train Loss: 0.1071 Acc: 0.9547


100%|██████████| 2/2 [00:15<00:00,  8.00s/it]

val Loss: 0.1080 Acc: 0.9608
